<a href="https://colab.research.google.com/github/RAVIPATISRIVIDYA/devtraining-needit-madrid/blob/master/extractionattacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
%matplotlib inline
import tensorflow_hub as thub
import tensorflow.keras.backend as K

In [2]:
!pip install adversarial-robustness-toolbox
import art

     |████████████████████████████████| 1.1 MB 6.4 MB/s 
     |████████████████████████████████| 3.3 MB 22.9 MB/s 
     |████████████████████████████████| 26.3 MB 90 kB/s 
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: numba
    Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2


In [3]:
from art.estimators.classification import KerasClassifier
from art.attacks.evasion import FastGradientMethod
from art.attacks.extraction import CopycatCNN, FunctionallyEquivalentExtraction, KnockoffNets

In [4]:
def sample_by_class(data, labels, num_samples=100):
    sample_data = []
    sample_labels = []
    unq_labels = list(range(labels.shape[1]))
    for label in unq_labels:
        idx = labels[:,label]==1
        sample_set = data[idx][0:num_samples].copy()
        label_set = labels[idx][0:num_samples].copy()
        sample_data.append(sample_set)
        sample_labels.append(label_set)
    
    sample_data = np.concatenate(sample_data)
    sample_labels = np.concatenate(sample_labels)
    print(sample_data.shape, sample_labels.shape)
    return sample_data, sample_labels

def subset_data(data, labels, fraction=5):
    data_size = data.shape[0]
    out_size = int(data_size*fraction/100)
    idx = np.random.choice(data_size, out_size, replace=False)
    out_data = data[idx].copy()
    out_labels = labels[idx].copy()
    
    print(out_data.shape, out_labels.shape)
    return out_data, out_labels

In [5]:
from tensorflow import keras
(x_train,y_train),(x_test,y_test) = keras.datasets.cifar10.load_data()

170508288/170498071 [==============================] - 3s 0us/step


In [6]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [7]:
testd = x_train[0:1000].copy()
testl = y_train[0:1000].copy()

In [8]:
def build_ganeval_model():
    K.clear_session()
    model_url = "https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1"
    ganeval_module = thub.Module(model_url)
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(32,32,3)))
    model.add(thub.KerasLayer(ganeval_module))
    model.add(tf.keras.layers.Activation('softmax'))
    
    model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.1, momentum=0.9, decay= 1e-4),
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=['accuracy'])
    
    return model

In [9]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
K.clear_session()
model_url = "https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1"
ganeval_module = thub.Module(model_url)

In [10]:
ge_cifar_clf = tf.keras.Sequential()
ge_cifar_clf.add(tf.keras.layers.InputLayer(input_shape=(32,32,3)))
# gan_eval_model.add(gan_eval_layer)
ge_cifar_clf.add(thub.KerasLayer(ganeval_module))
ge_cifar_clf.add(tf.keras.layers.Activation('softmax'))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [11]:
ge_cifar_clf.compile(optimizer=tf.keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=1e-4),
                       loss=tf.keras.losses.CategoricalCrossentropy(),
                       metrics=['accuracy'])
ge_cifar_clf.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 10)                7796426   
                                                                 
 activation (Activation)     (None, 10)                0         
                                                                 
Total params: 7,796,426
Trainable params: 0
Non-trainable params: 7,796,426
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [12]:
ge_cifar_clf = build_ganeval_model()
ge_cifar_clf.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 10)                7796426   
                                                                 
 activation (Activation)     (None, 10)                0         
                                                                 
Total params: 7,796,426
Trainable params: 0
Non-trainable params: 7,796,426
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [13]:
ge_cifar_clf.evaluate(testd, testl)

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


[166.622517578125, 0.11]

In [14]:
classifier = KerasClassifier(model=ge_cifar_clf, clip_values=(0, 1), use_logits=False)
attack_fgsm = FastGradientMethod(estimator=classifier, eps=0.4)

In [15]:
ge_cifar_clf.evaluate(testd, testl)

[166.622517578125, 0.11]

In [16]:
classifier = KerasClassifier(model=ge_cifar_clf, clip_values=(0, 1), use_logits=True)
attack_fgsm = FastGradientMethod(estimator=classifier, eps=0.4)

In [17]:
x_test_adv = attack_fgsm.generate(testd.astype(np.float32).copy())

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [18]:
ge_cifar_clf.evaluate(x_test_adv, testl)

[166.089640625, 0.11]

In [19]:
def build_substitute_model(enable_logits=True):
    model = tf.keras.Sequential( )
    model.add(Conv2D(32, (3, 3), padding="same", input_shape=(32,32,3)))
    model.add(Activation("relu"))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(keras.layers.Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation("relu"))
    model.add(Dropout(0.5))
    model.add(Dense(10))
    if enable_logits:
        model.add(Activation("linear"))
    else:
        model.add(Activation("softmax"))
        
    model.compile(loss=CategoricalCrossentropy(from_logits=enable_logits),
                  optimizer='adam',
                  metrics=['accuracy'] )
    
    return model

In [20]:
from tensorflow.keras.layers import Conv2D
from keras.layers import Activation, Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dropout,Flatten
import tensorflow as tf
from tensorflow.keras.losses import CategoricalCrossentropy
sub_model = build_substitute_model()
sub_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation_1 (Activation)   (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 activation_2 (Activation)   (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 15, 15, 32)        0         
                                                      

In [23]:
def extraction_attack(ART_attack, x_train, y_train, fractions, epoch=30, verbose=0):
    
    data_size = x_train.shape[0]
    epochs = epoch
    
    ### Train and Test sample data from each class to get final results
    print("Extract 100 Training samples from each class:")
    x_train_adv, y_train_adv = sample_by_class(x_train, y_train, num_samples=100)
    print("Extract 100 Test samples from each class:")
    x_test_adv, y_test_adv = sample_by_class(x_test, y_test, num_samples=100)
    
    ge_cifar_clf = build_ganeval_model()
    cloud_art_clf = KerasClassifier(model=ge_cifar_clf, clip_values=(0, 1), use_logits=True)
    
    loss1, acc1 = cloud_art_clf._model.evaluate(x_train_adv, y_train_adv, verbose=verbose)
    loss2, acc2 = cloud_art_clf._model.evaluate(x_test_adv, y_test_adv, verbose=verbose)
    
    clf_results = []
    
    for each_frac in fractions:
        max_queries = int(data_size*each_frac/100)
        print("Attacking the victim with %d percent of training data: %d queries..."%(each_frac, max_queries))
#         partial_x_train, partial_y_train = subset_data(x_train, y_train, fraction=each_frac)
        
        sub_clf = build_substitute_model()
        stolen_clf = KerasClassifier(model=sub_clf, clip_values=(0, 1), use_logits=True)
        
        attack = ART_attack(classifier=cloud_art_clf, 
                        batch_size_fit=32, 
                        batch_size_query=32,
                        nb_epochs=epochs,
                        nb_stolen=max_queries)
        
        stolen_clf = attack.extract(x_train, thieved_classifier=stolen_clf)

        loss3, acc3 = stolen_clf._model.evaluate(x_train_adv, y_train_adv, verbose=verbose)
        loss4, acc4 = stolen_clf._model.evaluate(x_test_adv, y_test_adv, verbose=verbose)
        
        clf_results.append((ART_attack.__name__, each_frac, max_queries, epochs,
                            (round(loss1,3), acc1), (round(loss2,3), acc2), 
                            (round(loss3,3), acc3), (round(loss4,3), acc4)))
        
        results_df = pd.DataFrame(clf_results, columns=['Attack', 'Amount of data', 'Epochs','Queries needed', 
                                                         'Victim results on Train data',
                                                       'Victim results on Test data', 'Clone model results on Train data', 
                                                        'Clone model results on Test data'])
    return results_df

In [24]:
import pandas as pd
KON_results_20e = extraction_attack(KnockoffNets, x_train, y_train, [5,10,30], epoch=30)
KON_results_30e = extraction_attack(KnockoffNets, x_train, y_train, [5,10,30], epoch=10)
KON_results_40e = extraction_attack(KnockoffNets, x_train, y_train, [5,10,30], epoch=35)

Extract 100 Training samples from each class:
(1000, 32, 32, 3) (1000, 10)
Extract 100 Test samples from each class:
(1000, 32, 32, 3) (1000, 10)
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


Attacking the victim with 5 percent of training data: 2500 queries...


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Attacking the victim with 10 percent of training data: 5000 queries...
Attacking the victim with 30 percent of training data: 15000 queries...
Extract 100 Training samples from each class:
(1000, 32, 32, 3) (1000, 10)
Extract 100 Test samples from each class:
(1000, 32, 32, 3) (1000, 10)
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Attacking the victim with 5 percent of training data: 2500 queries...
Attacking the victim with 10 percent of training data: 5000 queries...
Attacking the victim with 30 percent of training data: 15000 queries...
Extract 100 Training samples from each class:
(1000, 32, 32, 3) (1000, 10)
Extract 100 Test samples from each class:
(1000, 32, 32, 3) (1000, 10)
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Attacking the victim with 5 percent of training data: 2500 queries...
Attacking the victim with 10 percent of training data: 5000 queries...
Attacking the victim with 30 percent of training data: 15000 queries...


In [ ]:
CCC_results_30e = extraction_attack(CopycatCNN, x_train, y_train, [5,10,30], epoch=30)

In [26]:
EA_results = pd.concat([KON_results_20e,KON_results_30e,KON_results_40e])
EA_results.head(20)

,Attack,Amount of data,Epochs,Queries needed,Victim results on Train data,Victim results on Test data,Clone model results on Train data,Clone model results on Test data
0,KnockoffNets,5,2500,30,"(170.88, 0.098)","(170.812, 0.101)","(19.704, 0.102)","(18.824, 0.103)"
1,KnockoffNets,10,5000,30,"(170.88, 0.098)","(170.812, 0.101)","(25.059, 0.1)","(23.716, 0.099)"
2,KnockoffNets,30,15000,30,"(170.88, 0.098)","(170.812, 0.101)","(27.243, 0.101)","(25.768, 0.101)"
0,KnockoffNets,5,2500,10,"(170.88, 0.098)","(170.812, 0.101)","(12.253, 0.1)","(12.218, 0.1)"
1,KnockoffNets,10,5000,10,"(170.88, 0.098)","(170.812, 0.101)","(10.918, 0.101)","(10.651, 0.102)"
2,KnockoffNets,30,15000,10,"(170.88, 0.098)","(170.812, 0.101)","(16.862, 0.1)","(16.012, 0.1)"
0,KnockoffNets,5,2500,35,"(170.88, 0.098)","(170.812, 0.101)","(18.995, 0.096)","(18.983, 0.098)"
1,KnockoffNets,10,5000,35,"(170.88, 0.098)","(170.812, 0.101)","(19.094, 0.102)","(18.159, 0.101)"
2,KnockoffNets,30,15000,35,"(170.88, 0.098)","(170.812, 0.101)","(26.524, 0.099)","(25.221, 0.099)"


In [29]:
EA_results = pd.concat([CCC_results_30e])
EA_results.head(10)

,Attack,Amount of data,Epochs,Queries needed,Victim results on Train data,Victim results on Test data,Clone model results on Train data,Clone model results on Test data
0,CopycatCNN,5,2500,30,"(170.88, 0.098)","(170.812, 0.101)","(21.248, 0.102)","(19.783, 0.098)"
1,CopycatCNN,10,5000,30,"(170.88, 0.098)","(170.812, 0.101)","(24.865, 0.098)","(23.475, 0.1)"
2,CopycatCNN,30,15000,30,"(170.88, 0.098)","(170.812, 0.101)","(27.923, 0.099)","(26.533, 0.1)"


In [30]:
def CCC_extraction_attack(npd_data, x_train, y_train, fractions, epoch=30, verbose=0):
    
    data_size = x_train.shape[0]
    epochs = epoch
    npd_size = npd_data.shape[0]
    
    ### Train and Test sample data from each class to get final results
    print("Extract 100 Training samples from each class:")
    x_train_adv, y_train_adv = sample_by_class(x_train, y_train, num_samples=100)
    print("Extract 100 Test samples from each class:")
    x_test_adv, y_test_adv = sample_by_class(x_test, y_test, num_samples=100)
    
    ge_cifar_clf = build_ganeval_model()
    cloud_art_clf = KerasClassifier(model=ge_cifar_clf, clip_values=(0, 1), use_logits=True)
    
    loss1, acc1 = cloud_art_clf._model.evaluate(x_train_adv, y_train_adv, verbose=verbose)
    loss2, acc2 = cloud_art_clf._model.evaluate(x_test_adv, y_test_adv, verbose=verbose)
            
    sub_clf = build_substitute_model()
    stolen_clf = KerasClassifier(model=sub_clf, clip_values=(0, 1), use_logits=True)
    
    attack = CopycatCNN(classifier=cloud_art_clf, 
                        batch_size_fit=32, 
                        batch_size_query=32,
                        nb_epochs=epochs,
                        nb_stolen=npd_size)
    
    stolen_clf = attack.extract(npd_data, thieved_classifier=stolen_clf)
    
    stolen_clf._model.save_weights("CCC_temp.h5")
    
    clf_results = []
    
    for each_frac in fractions:

        max_queries = int(data_size*each_frac/100)
        print("Attacking the victim with %d percent of training data: %d queries..."%(each_frac, max_queries))
        
        stolen_clf._model.load_weights("CCC_temp.h5")
        partial_x_train, _ = subset_data(x_train, y_train, fraction=each_frac)

        stolen_clf = attack.extract(partial_x_train, thieved_classifier=stolen_clf)
        
        loss3, acc3 = stolen_clf._model.evaluate(x_train_adv, y_train_adv, verbose=verbose)
        loss4, acc4 = stolen_clf._model.evaluate(x_test_adv, y_test_adv, verbose=verbose)
        
        clf_results.append(("Copycat CNN", npd_size, each_frac, max_queries, epochs, 
                            (round(loss1,3), acc1), 
                            (round(loss2,3), acc2), 
                            (round(loss3,3), acc3), 
                            (round(loss4,3), acc4)))
        
        results_df = pd.DataFrame(clf_results, columns=['Attack', "NPD Queries", 'Fraction of train data', 
                                                        'Train queries', 
                                                        'Epochs trained', 'Victim clf on Train data',
                                                       'Victim clf on Test data', 'Stolen clf on Train data', 
                                                        'Stolen clf on Test data'])
    return results_df

In [31]:
from tensorflow.keras.datasets import cifar100
(cifar100_x, cifar100_y), _ = cifar100.load_data(label_mode='fine')
print('cifar100 training data shape:', cifar100_x.shape)

# Normalize pixel values
cifar100_x = cifar100_x/255

# Sample 20,000 data points randomly from the data
idx = np.random.choice(cifar100_x.shape[0], 20000, replace=False)
cifar100_x = cifar100_x[idx].copy()
cifar100_y = cifar100_y[idx].copy()

print('Final NPD size of cifar100 training data:', cifar100_x.shape)

169017344/169001437 [==============================] - 3s 0us/step
cifar100 training data shape: (50000, 32, 32, 3)
Final NPD size of cifar100 training data: (20000, 32, 32, 3)


In [32]:
CCC_results_30e = CCC_extraction_attack(cifar100_x, x_train, y_train, [5,10,30], epoch=30)

Extract 100 Training samples from each class:
(1000, 32, 32, 3) (1000, 10)
Extract 100 Test samples from each class:
(1000, 32, 32, 3) (1000, 10)
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Train on 20000 samples
Epoch 1/30
20000/20000 [==============================] - 9s 457us/sample - loss: 2.0761 - accuracy: 0.2313
Epoch 2/30
20000/20000 [==============================] - 9s 448us/sample - loss: 1.9525 - accuracy: 0.2813
Epoch 3/30
20000/20000 [==============================] - 9s 462us/sample - loss: 1.8846 - accuracy: 0.3091
Epoch 4/30
20000/20000 [==============================] - 12s 593us/sample - loss: 1.8144 - accuracy: 0.3335
Epoch 5/30
20000/20000 [==============================] - 12s 602us/sample - loss: 1.7576 - accuracy: 0.3537
Epoch 6/30
20000/20000 [==============================] - 12s 589us/sample - loss: 1.7045 - accuracy: 0.3679
Epoch 7/30
20000/20000 [==============================] - 12s 575us/sample - loss: 1.6526 - accuracy: 0.3895
Epoch 8/30
20000/20000 [==============================] - 11s 572us/sample - loss: 1.6095 - accuracy: 0.4099
Epoch 9/30
20000/20000 [==============================] - 12s 576us/sample - loss: 1.5640 - accuracy: 0.4232

(2500, 32, 32, 3) (2500, 10)
Train on 2500 samples
Epoch 1/30
2500/2500 [==============================] - 1s 436us/sample - loss: 31.2618 - accuracy: 0.8280
Epoch 2/30
2500/2500 [==============================] - 1s 412us/sample - loss: 1.8237 - accuracy: 0.8544
Epoch 3/30
2500/2500 [==============================] - 1s 423us/sample - loss: 0.8788 - accuracy: 0.8780
Epoch 4/30
2500/2500 [==============================] - 1s 427us/sample - loss: 0.6731 - accuracy: 0.8992
Epoch 5/30
2500/2500 [==============================] - 1s 447us/sample - loss: 0.5644 - accuracy: 0.9156
Epoch 6/30
2500/2500 [==============================] - 1s 446us/sample - loss: 0.4417 - accuracy: 0.9200
Epoch 7/30
2500/2500 [==============================] - 1s 453us/sample - loss: 0.3591 - accuracy: 0.9236
Epoch 8/30
2500/2500 [==============================] - 1s 448us/sample - loss: 0.3598 - accuracy: 0.9236
Epoch 9/30
2500/2500 [==============================] - 1s 460us/sample - loss: 0.3672 - accuracy: 0

Attacking the victim with 10 percent of training data: 5000 queries...
(5000, 32, 32, 3) (5000, 10)
Train on 5000 samples
Epoch 1/30
5000/5000 [==============================] - 2s 439us/sample - loss: 19.5105 - accuracy: 0.8404
Epoch 2/30
5000/5000 [==============================] - 2s 433us/sample - loss: 0.8131 - accuracy: 0.8764
Epoch 3/30
5000/5000 [==============================] - 2s 434us/sample - loss: 0.6375 - accuracy: 0.9068
Epoch 4/30
5000/5000 [==============================] - 2s 440us/sample - loss: 0.5075 - accuracy: 0.9200
Epoch 5/30
5000/5000 [==============================] - 2s 423us/sample - loss: 0.4587 - accuracy: 0.9246
Epoch 6/30
5000/5000 [==============================] - 2s 439us/sample - loss: 0.4283 - accuracy: 0.9264
Epoch 7/30
5000/5000 [==============================] - 2s 445us/sample - loss: 0.3912 - accuracy: 0.9266
Epoch 8/30
5000/5000 [==============================] - 2s 445us/sample - loss: 0.3690 - accuracy: 0.9268
Epoch 9/30
5000/5000 [=======

Attacking the victim with 30 percent of training data: 15000 queries...
(15000, 32, 32, 3) (15000, 10)
Train on 15000 samples
Epoch 1/30
15000/15000 [==============================] - 7s 441us/sample - loss: 6.0331 - accuracy: 0.8691
Epoch 2/30
15000/15000 [==============================] - 7s 461us/sample - loss: 0.5243 - accuracy: 0.9253
Epoch 3/30
15000/15000 [==============================] - 7s 474us/sample - loss: 0.4176 - accuracy: 0.9253
Epoch 4/30
15000/15000 [==============================] - 7s 471us/sample - loss: 0.3856 - accuracy: 0.9254
Epoch 5/30
15000/15000 [==============================] - 7s 462us/sample - loss: 0.3708 - accuracy: 0.9259
Epoch 6/30
15000/15000 [==============================] - 7s 473us/sample - loss: 0.3532 - accuracy: 0.9257
Epoch 7/30
15000/15000 [==============================] - 7s 467us/sample - loss: 0.3441 - accuracy: 0.9257
Epoch 8/30
15000/15000 [==============================] - 7s 471us/sample - loss: 0.3463 - accuracy: 0.9256
Epoch 9/30

In [34]:
CCC_results = pd.concat([CCC_results_30e])
CCC_results.head(24)

,Attack,NPD Queries,Fraction of train data,Train queries,Epochs trained,Victim clf on Train data,Victim clf on Test data,Stolen clf on Train data,Stolen clf on Test data
0,Copycat CNN,20000,5,2500,30,"(170.88, 0.098)","(170.812, 0.101)","(8.612, 0.1)","(8.445, 0.1)"
1,Copycat CNN,20000,10,5000,30,"(170.88, 0.098)","(170.812, 0.101)","(30.454, 0.1)","(30.825, 0.1)"
2,Copycat CNN,20000,30,15000,30,"(170.88, 0.098)","(170.812, 0.101)","(6.978, 0.1)","(6.978, 0.1)"


In [35]:
x_train_adv, y_train_adv = sample_by_class(x_train, y_train, num_samples=100)
x_test_adv, y_test_adv = sample_by_class(x_test, y_test, num_samples=100)

(1000, 32, 32, 3) (1000, 10)
(1000, 32, 32, 3) (1000, 10)


In [39]:
partial_x_train, partial_y_train = subset_data(x_train, y_train, fraction=0.3)

partial_x_train = partial_x_train.reshape(-1,32*32*3)

### Tried even with standardizing the values around '0'
x_mean = partial_x_train.mean()
x_std = partial_x_train.std()
# partial_x_train = (partial_x_train - x_mean)/x_std

partial_x_train.shape

(150, 32, 32, 3) (150, 10)


(150, 3072)

In [40]:
from keras.models import Sequential
fen_eval_model = Sequential()
fen_eval_model.add(Dense(1024, activation="relu", input_shape=(32*32*3,)))
fen_eval_model.add(Dense(10, activation="linear"))

fen_eval_model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001,),
    metrics=["accuracy"],
)

In [41]:
fen_eval_model.fit(x_train.reshape(-1,32*32*3), y_train, batch_size=32, epochs=30, verbose=1)

Train on 50000 samples
Epoch 1/30
50000/50000 [==============================] - 8s 167us/sample - loss: 30.0469 - accuracy: 0.2512
Epoch 2/30
50000/50000 [==============================] - 8s 165us/sample - loss: 13.6426 - accuracy: 0.2796
Epoch 3/30
50000/50000 [==============================] - 8s 163us/sample - loss: 9.0106 - accuracy: 0.2999
Epoch 4/30
50000/50000 [==============================] - 8s 163us/sample - loss: 6.0566 - accuracy: 0.3033
Epoch 5/30
50000/50000 [==============================] - 8s 164us/sample - loss: 3.0482 - accuracy: 0.2352
Epoch 6/30
50000/50000 [==============================] - 8s 165us/sample - loss: 2.0673 - accuracy: 0.2635
Epoch 7/30
50000/50000 [==============================] - 8s 163us/sample - loss: 1.9065 - accuracy: 0.2971
Epoch 8/30
50000/50000 [==============================] - 8s 161us/sample - loss: 1.8446 - accuracy: 0.3118
Epoch 9/30
50000/50000 [==============================] - 8s 163us/sample - loss: 1.8196 - accuracy: 0.3164
Epo

In [42]:
fen_eval_model.evaluate(x_test.reshape(-1,32*32*3), y_test)

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


[1.7292973426818847, 0.3781]

In [43]:
fen_eval_model.evaluate(partial_x_train, partial_y_train)

[1.5711732721328735, 0.42]

In [44]:
flat_art_clf = KerasClassifier(model=fen_eval_model, clip_values=(0, 1), use_logits=False)

attack_FEN = FunctionallyEquivalentExtraction(classifier=flat_art_clf, num_neurons=1024)
stolen_clf_FEN = attack_FEN.extract(partial_x_train, partial_y_train,
                                    delta_0=0.06,
                                    fraction_true=0.3,
                                    rel_diff_slope=0.1,
                                    rel_diff_value=0.1,
                                    delta_init_value=0.1,
                                    delta_value_max=2,
                                    d2_min=0.04,
                                    d_step=0.01,
                                    delta_sign=0.2,
                                    unit_vector_scale=1000)

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
/usr/local/lib/python3.7/dist-packages/art/attacks/extraction/functionally_equivalent_extraction.py:221: RuntimeWarning: divide by zero encountered in true_divide
  if np.sum(np.abs((m_1 - m_2) / m_1) < rel_diff_slope) > fraction_true * self.num_classes:
/usr/local/lib/python3.7/dist-packages/art/attacks/extraction/functionally_equivalent_extraction.py:221: RuntimeWarning: invalid value encountered in true_divide
  if np.sum(np.abs((m_1 - m_2) / m_1) < rel_diff_slope) > fraction_true * self.num_classes:
/usr/local/lib/python3.7/dist-packages/art/attacks/extraction/functionally_equivalent_extraction.py:225: RuntimeWarning: divide by zero encountered in true_divide
  t_hat = t_1 + np.divide(y_2 - y_1 - (t_2 - t_1) * m_2, m_1 

KeyboardInterrupt: ignored

In [59]:
def build_ganeval_model_flat():
    K.clear_session()
    model_url = "https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1"
    ganeval_module = thub.Module(model_url)
    
    model = Sequential()
#     model.add(InputLayer(input_shape=(32*32*3,)))
    model.add(model.reshape((32,32,3),input_shape=(32*32*3,)))
    model.add(thub.KerasLayer(ganeval_module))
    model.add(Activation('relu'))
        
    model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=1e-4),
                  loss=CategoricalCrossentropy(),
                  metrics=['accuracy'])
    
    return model

In [60]:
ge_flat_clf = build_ganeval_model_flat()
ge_flat_clf.summary()

AttributeError: ignored

In [47]:
ge_flat_clf.evaluate(partial_x_train, partial_y_train)

NameError: ignored

In [48]:
flat_art_clf = KerasClassifier(model=ge_flat_clf, clip_values=(0, 1), use_logits=False)

attack_FEN = FunctionallyEquivalentExtraction(classifier=flat_art_clf, num_neurons=100)
stolen_clf_FEN = attack_FEN.extract(partial_x_train, partial_y_train,
                                    delta_0=0.05,
                                    fraction_true=0.3,
                                    rel_diff_slope=0.1,
                                    rel_diff_value=0.1,
                                    delta_init_value=0.1,
                                    delta_value_max=2,
                                    d2_min=0.04,
                                    d_step=0.01,
                                    delta_sign=0.2,
                                    unit_vector_scale=1000)

NameError: ignored

In [ ]:
stolen_clf_FEN._model.evaluate(partial_x_train, partial_y_train)

### Evaluation
stolen_clf_FEN._model.evaluate(x_train_adv, y_train_adv)
stolen_clf_FEN._model.evaluate(x_test_adv, y_test_adv)